# Tutorial: Mel-Frequency Cepstral Coefficients (MFCCs)

Neste notebook mostraremos 

Caso não queira ver os códigos e funções utilizados clique
[aqui](#Passo-a-passo-da-obtenção-de-MFCCs)
para ir direto para a exibição das etapas.

## Importação de bibliotecas e leitura do sinal

Foram utilizados recursos das seguintes bibliotecas:

- [numpy](https://numpy.org/doc/stable/user/whatisnumpy.html)
- [scipy](https://docs.scipy.org/doc/scipy/reference/)
- [librosa](https://librosa.org/doc/latest/index.html)
- [matplotlib](https://matplotlib.org/stable/api/index)
- [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/)

O áudio utilizado para construção da MFCC neste notebook foi retirado do Open Speech Repository, que possui vários arquivos de voz gratuitos em diversos idiomas. Vece pode acessá-lo clicando [aqui](http://www.voiptroubleshooter.com/open_speech/index.html).

In [1]:
import numpy
import numpy as np
import scipy.signal, scipy.io.wavfile
from   scipy.fftpack import dct
import matplotlib.pyplot as plt, matplotlib.cm as cm, IPython.display as ipd
import ipywidgets as widgets
from   ipywidgets import interact, interactive, fixed, interact_manual
import librosa, librosa.display

sample_rate, signal = scipy.io.wavfile.read('./audio/OSR_us_000_0011_8k.wav')
signal = signal[int(0.5 * sample_rate):int(3.5 * sample_rate)]  # Utilizar somente os primeiros segundos

## Obtenção das MFCCs


### Parâmetros

As variáveis abaixo são utilizadas para configurar a forma como são calculados e exibidos os coeficientes. Recomendamos manter os valores pre-estabelesdos para uma melhor visualização. 

In [2]:
pre_emphasis = 0.97
frame_size = 0.025
frame_stride = 0.01
NFFT = 512
nfilt = 40
num_ceps = 12
cep_lifter = 22

### Cálculos
Nesta célula são realizados todos os cálculos que darão origem aos dados apresentados na exibição ao fim deste notebook.

In [3]:
#-----------------------------------------------------------------------------------------------------------------------------

# Pre-Emphasis
emphasized_signal = numpy.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])

#-----------------------------------------------------------------------------------------------------------------------------

# Framing
frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
signal_length = len(emphasized_signal)
frame_length = int(round(frame_length))
frame_step = int(round(frame_step))
num_frames = int(numpy.ceil(float(numpy.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame
pad_signal_length = num_frames * frame_step + frame_length
z = numpy.zeros((pad_signal_length - signal_length))
pad_signal = numpy.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal
indices = numpy.tile(numpy.arange(0, frame_length), (num_frames, 1)) + numpy.tile(numpy.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
frames_ret = pad_signal[indices.astype(numpy.int32, copy=False)]

#-----------------------------------------------------------------------------------------------------------------------------

# Window
frames = frames_ret * numpy.hamming(frame_length)
# frames *= 0.54 - 0.46 * numpy.cos((2 * numpy.pi * n) / (frame_length - 1))  # Explicit Implementation **

#-----------------------------------------------------------------------------------------------------------------------------

# Fourier-Transform and Power Spectrum
mag_frames = numpy.absolute(numpy.fft.rfft(frames, NFFT))  # Magnitude of the FFT
pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum

#-----------------------------------------------------------------------------------------------------------------------------

# Filter Banks
low_freq_mel = 0
high_freq_mel = (2595 * numpy.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
mel_points = numpy.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
bin = numpy.floor((NFFT + 1) * hz_points / sample_rate)
fbank = numpy.zeros((nfilt, int(numpy.floor(NFFT / 2 + 1))))
for m in range(1, nfilt + 1):
    f_m_minus = int(bin[m - 1])   # left
    f_m = int(bin[m])             # center
    f_m_plus = int(bin[m + 1])    # right

    for k in range(f_m_minus, f_m):
        fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
    for k in range(f_m, f_m_plus):
        fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
filter_banks = numpy.dot(pow_frames, fbank.T)
filter_banks = numpy.where(filter_banks == 0, numpy.finfo(float).eps, filter_banks)  # Numerical Stability
filter_banks = 20 * numpy.log10(filter_banks)  # dB

#-----------------------------------------------------------------------------------------------------------------------------

# Mel-frequency Cepstral Coefficients (MFCCs)
mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # Keep 2-13
delta = librosa.feature.delta(mfcc)
delta_delta = librosa.feature.delta(mfcc, order=2)
(nframes, ncoeff) = mfcc.shape
n = numpy.arange(ncoeff)
lift = 1 + (cep_lifter / 2) * numpy.sin(numpy.pi * n / cep_lifter)
mfcc *= lift  #*

#-----------------------------------------------------------------------------------------------------------------------------

# Mean Normalization
filter_banks_norm = filter_banks - (numpy.mean(filter_banks, axis=0) + 1e-8)
mfcc_norm = mfcc - (numpy.mean(mfcc, axis=0) + 1e-8)

#-----------------------------------------------------------------------------------------------------------------------------


---
## Gráficos


### Funções para a plotagem dos gráficos

Nesta cécula foram defnidas várias funções para realizar a exibição das informações 

In [6]:
def plotSound(audio, sr, title='Sinal no domínio do tempo'):
    N = len(audio)
    T = N/sr
    t = np.linspace(0,T,N)
    plt.figure(figsize=(12,2.5));
    plt.plot(t,audio);
    plt.title(title, fontsize=18)
    plt.xlabel('tempo (s)', fontsize=15); plt.ylabel('Amplitude', fontsize=15);
    plt.grid(which='both')
    plt.show()
    ipd.display(ipd.Audio(audio, rate=sr, autoplay=False))

def PreEnfase(signal,emphasized_signal, sr):
    text = open('./texts/pre-emphasis.md', mode='r', encoding='utf-8').read()

    ipd.display_markdown(text, raw=True)
    plotSound(signal, sr, title='Sinal original no domínio do tempo')
    plotSound(emphasized_signal, sr, title='Sinal enfatizado no domínio do tempo')
    
def plotWindow(frame_length):
    plt.figure(figsize=(7,5));
    plt.plot(numpy.hamming(frame_length));
    plt.title('Janela Hamming', fontsize=18)
    plt.xlabel('amostras', fontsize=15); plt.ylabel('Amplitude', fontsize=15);
    plt.grid(which='both', linestyle='dotted')
    plt.show()
    
def plotFrame(frame, title='Fragmento do sinal'):
    plt.figure(figsize=(7,5));
    plt.plot(frame);
    plt.title(title, fontsize=18)
    plt.xlabel('amostras', fontsize=15); plt.ylabel('Amplitude', fontsize=15);
    plt.grid(which='both', linestyle='dotted')
    plt.show()

def plotWindowed(frames, frame_length):
    text1 = open('./texts/windowed1.md', mode='r', encoding='utf-8').read()
    ipd.display_markdown(text1, raw=True)
    plotWindow(frame_length)
    nFrame=widgets.IntText(value=100, description='N° do frame:', disabled=False)
    text2 = open('./texts/windowed2.md', mode='r', encoding='utf-8').read()
    ipd.display_markdown(text2, raw=True)
    plotFrame(frames[int(nFrame.value)], title=f'Frame n° {nFrame.value} janelado')

def plotFrames(frames):
    text = open('./texts/framing.md', mode='r', encoding='utf-8').read()
    ipd.display_markdown(text, raw=True)
    nFrame=widgets.IntText(value=100, description='N° do frame:', disabled=False)
    plotFrame(frames[int(nFrame.value)], title=f'Frame n° {nFrame.value}')
        
def plotSpectogram(frames, option=None):
    if(option == 'Transformada de Fourier'):
        text = open('./texts/fourier-Transform.md', mode='r', encoding='utf-8').read()
        ipd.display_markdown(text, raw=True)
    elif(option == 'Espectro de Potência'):
        text = open('./texts/power-spectrum.md', mode='r', encoding='utf-8').read()
        ipd.display_markdown(text, raw=True)
    plt.figure(figsize=(12, 2.5))
    plt.imshow(np.flipud(frames.T), cmap=cm.jet, aspect=0.15, extent=[0,3,0,4]);
    plt.title('Espectrograma do Sinal', fontsize=18);
    plt.ylabel('Frequencia (kHz)', fontsize=15);
    plt.xlabel('tempo (s)', fontsize=15);
    plt.show();

def plotFBank(fbank, sample_rate, low_freq):
    plt.figure(figsize=(12,2.5))
    plt.title('Banco de filtros MEL', fontsize=18)
    plt.plot(np.linspace(low_freq, (sample_rate / 2), 257), fbank.T)
    plt.xlabel('Frequências (escala MEL)',fontsize=15); plt.ylabel('Amplitude', fontsize=15);
    plt.grid(which='both', linestyle='dotted')
    plt.show()
    plt.figure(figsize=(10, 3.5))
    librosa.display.specshow(fbank, sr=sample_rate, x_axis='linear', y_axis='frames')
    plt.colorbar()
    plt.title('Espectrograma do banco de filtros')
    plt.ylabel('N° do filtro')
    plt.tight_layout()
    plt.show()

def plotMFCC(num_ceps, mfcc):
    plt.figure(figsize=(12, 2.5))
    plt.imshow(np.flipud(mfcc.T), interpolation='nearest', cmap=cm.jet, origin='lower', 
               aspect=0.065, extent=[0,3.5,1,num_ceps])
    plt.title('MFCCs', fontsize=18)
    plt.xlabel('tempo (s)', fontsize=15); plt.ylabel('Coeficientes', fontsize=15);
    plt.show()
    # plt.savefig('./images/mfcc.png', bbox_inches='tight', dpi=200)

def Cepstrum(num_ceps, mfcc):
    text = open('./texts/cepstrum.md', mode='r', encoding='utf-8').read()
    ipd.display_markdown(text, raw=True)
    plotMFCC(num_ceps, mfcc)
    
def meanNorm(num_ceps, mfcc_norm,filter_banks_norm):
    plt.figure(figsize=(12, 2.5))
    plt.imshow(np.flipud(filter_banks_norm.T), cmap=cm.jet, aspect=0.00015, extent=[0,3,0,4000]);
    plt.title('Espectrograma Normalizado', fontsize=18);
    plt.ylabel('Frequencia (Mel)', fontsize=15);
    plt.xlabel('tempo (s)', fontsize=15);
    plt.show();
    plt.figure(figsize=(12, 2.5))
    plt.imshow(np.flipud(mfcc_norm.T), interpolation='nearest', cmap=cm.jet, origin='lower', 
               aspect=0.065, extent=[0,3.5,1,num_ceps])
    plt.title("MFCCs Normalizados", fontsize=18)
    plt.xlabel('tempo (s)', fontsize=15); plt.ylabel('Coeficientes', fontsize=15);
    plt.show()
    
def plotMFCC_librosa(num_ceps,mfcc):
    plt.figure(figsize=(15, 5))
    librosa.display.specshow(np.flipud(mfcc), x_axis='time', y_axis='frames')
    plt.colorbar()
    plt.title('MFCC')
    plt.tight_layout()
    plt.show()
    
def plotMEL(fbank, sample_rate, low_freq_mel, filter_banks):
    text = open('./texts/filter-Banks1.md', mode='r', encoding='utf-8').read()
    ipd.display_markdown(text, raw=True)
    plotFBank(fbank, sample_rate, low_freq_mel)
    text2 = open('./texts/filter-Banks2.md', mode='r', encoding='utf-8').read()
    ipd.display_markdown(text2, raw=True)
    plt.figure(figsize=(12, 2.5))
    plt.imshow(np.flipud(filter_banks.T), cmap=cm.jet, aspect=0.0002, extent=[0,3.5,0,4000]);
    plt.title('Espectrograma do Sinal', fontsize=18);
    plt.ylabel('Frequencia (Mel)', fontsize=15);
    plt.xlabel('tempo (s)', fontsize=15);
    plt.show();

def choosePlot(option=None):
    if option==None: print("Escolha uma etapa para visualisar.") 
    if option=='Pré-Enfase':             PreEnfase(signal,emphasized_signal, sample_rate)
    if option=='Fragmentação':           plotFrames(frames_ret)
    if option=='Janelamento':            plotWindowed(frames, frame_length)
    if option=='Transformada de Fourier':plotSpectogram(mag_frames, 'Transformada de Fourier')
    if option=='Espectro de Potência':   plotSpectogram(pow_frames, 'Espectro de Potência')
    if option=='Banco de filtros MEL':   plotMEL(fbank, sample_rate, low_freq_mel, filter_banks)
    if option=='Cepstrum':               Cepstrum(num_ceps,mfcc)
    if option=='Normalização de Média':  meanNorm(num_ceps, mfcc_norm, filter_banks_norm)

# Passo a passo da obtenção de MFCCs

In [5]:
Lista = ['Pré-Enfase','Fragmentação','Janelamento','Transformada de Fourier','Espectro de Potência','Banco de filtros MEL','Cepstrum','Normalização de Média']

interact(choosePlot, option=widgets.Dropdown(options = Lista, 
                                             description='Etapa:', value=None,
                                             disabled=False), continuous_update=False);

interactive(children=(Dropdown(description='Etapa:', options=('Pré-Enfase', 'Fragmentação', 'Janelamento', 'Tr…

# Referências

Haytham M. Fayek. (2016). Speech Processing for Machine Learning: Filter banks, Mel-Frequency Cepstral Coefficients (MFCCs) and What's In-Between.